# Initialize

In [73]:
from openai import OpenAI
from google import genai
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import ast
import time
from dotenv import load_dotenv
import os

load_dotenv()

my_key = os.getenv("GOOGLE_CLOUD_KEY_2")

# client = OpenAI(api_key=my_key)
client = genai.Client(api_key=my_key)

# Prompting prep

In [28]:
# Load dataset
df = pd.read_csv("../Data/hate_speech_and_offensive_language_dataset/labeled_data.csv")

# Sample some examples for context
df2 = df.copy()
df2['message'] = df2['tweet']
df2['labels'] = df2['class'].apply(lambda x: 0 if x == 2 else 1)

sample_size = 1000
n_zero = int(sample_size * 0.8)
n_one = sample_size - n_zero

df_zero = df2[df2['labels'] == 0]
df_one = df2[df2['labels'] == 1]

# if there aren't enough examples in a group, sample with replacement
replace_zero = len(df_zero) < n_zero
replace_one = len(df_one) < n_one

sample_zero = df_zero.sample(n=n_zero, random_state=42, replace=replace_zero)
sample_one = df_one.sample(n=n_one, random_state=42, replace=replace_one)

# API Calling

In [ ]:
# Format examples as text for the prompt
# example_text = "\n".join(
#     [f"Message: {row['message']}\nLabel: {row['labels']}" for _, row in examples.iterrows()]
# )

# # Create improved synthesis prompt
# prompt = f"""
# You are an expert data generator specializing in creating realistic chat messages for training
# an offensive language detection model.

# Below are examples of real data (Message and Label pairs).
# Label '1' means the message is offensive, and label '0' means it is not offensive.

# Examples:
# {example_text}

# Now, generate 5000 new synthetic examples that follow the same style, tone, and balance between classes.
# Try to make the data realistic — include slang, emojis, mild and strong insults, as well as casual and polite non-offensive chats.
# Avoid repetition or overly similar phrasing.

# Output your response **strictly** in this format (CSV-like, no extra commentary):

# Message,Label
# <message_1>,<label>
# <message_2>,<label>
# ...
# <message_20>,<label>
# """

In [74]:
num_batches = 100
samples_per_batch = 50
ratio_non_off = 0.8
ratio_off = 0.2

count_zero_per_batch = int(samples_per_batch * ratio_non_off)
count_one_per_batch  = int(samples_per_batch * ratio_off)

batches = []
for i in range(num_batches):
    start_idx_zero = i * count_zero_per_batch
    end_idx_zero = start_idx_zero + count_zero_per_batch
    start_idx_one = i * count_one_per_batch
    end_idx_one = start_idx_one + count_one_per_batch
    
    batch_zero = sample_zero.iloc[start_idx_zero:end_idx_zero]
    batch_one  = sample_one.iloc[start_idx_one:end_idx_one]
    
    batch = pd.concat([batch_zero, batch_one]).sample(frac=1, random_state=i).reset_index(drop=True)
    batches.append(batch)

In [75]:
results = []
for i, batch in tqdm(enumerate(batches), total=len(batches), desc="Processing Batches"):
    example_text = "\n".join([
        f"Message: {row['message']}\nLabel: {row['labels']}"
        for _, row in batch.iterrows()
    ])
    
    prompt = f"""
    You are generating labeled twitch chat messages for an offensive language detection model.
    Here are some examples (80% non-offensive, 20% offensive):
    {example_text}

    These examples are from twitter, but I want you to try making the generated content imitate a platform behavior from Twitch chat.
    So include typical Twitch behavior such as emotes, short phrases, hype words, and slang.

    Generate 50 new examples following the same 80:20 ratio:
    - 40 non-offensive messages
    - 10 offensive messages

    Output strictly as CSV with columns message,labels:
    - Use a comma to separate the message from its label
    - Do NOT include headers
    - Do NOT include ',' in the messages themselves
    - Each line should contain only the message and its label
    - The label should be 0 for non-offensive and 1 for offensive messages
    - Use the format specified below

    Here is the format:
    <message_1>,<label>
    <message_2>,<label>
    ...
    <message_50>,<label>

    Do not add any extra text or explanation, just output the CSV lines.
    """

    for attempt in range(5):
        try:
            response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt,
                config={"temperature": 0.9}
            )
            break  # success
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
            time.sleep(5)  # wait 5 seconds before retry
    results.append(response.text)
    print(f"Batch {i+1} complete.")

Processing Batches:   1%|          | 1/100 [00:04<07:14,  4.39s/it]

Batch 1 complete.


Processing Batches:   2%|▏         | 2/100 [00:08<07:01,  4.30s/it]

Batch 2 complete.


Processing Batches:   3%|▎         | 3/100 [00:13<07:39,  4.74s/it]

...

Batch 98 complete.


Processing Batches:  99%|█████████▉| 99/100 [07:22<00:03,  3.85s/it]

Batch 99 complete.


Processing Batches: 100%|██████████| 100/100 [07:26<00:00,  4.47s/it]

Batch 100 complete.


In [ ]:
with open("../Data/synthesized/synthesized_data_alt_fixed.csv", "w", encoding="utf-8") as f:
    f.write("message,label\n")
    for res in results:
        # Each res is multiple lines
        for line in res.splitlines():
            if not line.strip():  # skip empty lines
                continue
            try:
                message, label = line.rsplit(",", 1)  # split only on last comma
            except ValueError:
                continue  # skip malformed line
            # Escape inner quotes in message
            message = message.replace('"', '""')
            f.write(f'"{message}",{label}\n')

In [92]:
test = pd.read_csv("synthesized_data_alt_fixed.csv")

In [93]:
test.head(50)

,message,labels
0,PogChamp sheesh that play was insane!,0
1,LUL that fail!,0
2,FeelsBadMan they almost had it!,0
3,Hype! Hype! Hype!,0
4,ResidentSleeper zzzzzz,0
5,4Head that's what he gets,0
6,OMEGALUL,0
7,EZ Clap,0
8,KEKW,0
9,monkaS,0


In [87]:
test['Label'].value_counts()

Label
0                                                                     4788
1                                                                     1006
 maintaining an 80:20 ratio of non-offensive to offensive content.       1
Name: count, dtype: int64